<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">Classification with Python</font></h1>

In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

Lets download the dataset

In [2]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

--2021-03-18 17:56:22--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0.002s  

2021-03-18 17:56:22 (14.2 MB/s) - ‘loan_train.csv’ saved [23101/23101]



### Load Data From CSV File  

In [3]:
df = pd.read_csv('loan_train.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


In [4]:
df.shape

(346, 10)

### Convert to date time object 

In [5]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,male
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,female
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,male


# Data visualization and pre-processing



Let’s see how many of each class is in our data set 

In [6]:
df['loan_status'].value_counts()

PAIDOFF       260
COLLECTION     86
Name: loan_status, dtype: int64

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:

In [ ]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction

### Lets look at the day of the week people get the loan 

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values

Lets look at gender:

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay their loans while only 73 % of males pay their loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding  
#### How about education?

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature before One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical variables to binary variables and append them to the feature Data Frame 

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature selection

Lets defind feature sets, X:

In [ ]:
X = Feature
X[0:5]

What are our lables?

In [ ]:
y = df['loan_status'].values
y[0:5]

## Normalize Data 

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification 

Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



__ Notice:__ 
- You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
- You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
- You should include the code of the algorithm in the following cells.

#### Importing common libraries

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, jaccard_score

# K Nearest Neighbor(KNN)
Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the __loan_test.csv__ for finding the best k, however, you can split your train_loan.csv into train and test to find the best __k__.

#### splitting the data

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Validation set:', X_eval.shape,  y_eval.shape)

#### training (with k=6)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k = 6
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

#### predicting

In [ ]:
y_pred = neigh.predict(X_eval)
y_pred[0:5]

#### evaluating

In [ ]:
print (classification_report(y_eval, y_pred))

In [ ]:
print("Train set Accuracy: ", accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", accuracy_score(y_eval, y_pred))

#### Confusion Matrix

In [ ]:
confusion_matrix(y_eval, y_pred)

#### finding the best k

In [ ]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))

for n in range(1,Ks): 
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    y_pred=neigh.predict(X_eval)
    mean_acc[n-1] = accuracy_score(y_eval, y_pred)
    std_acc[n-1]=np.std(y_pred==y_eval)/np.sqrt(y_pred.shape[0])
mean_acc

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.fill_between(range(1,Ks),mean_acc - 3 * std_acc,mean_acc + 3 * std_acc, alpha=0.10,color="green")
plt.legend(('Accuracy ', '+/- 1xstd','+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbors (K)')
plt.tight_layout()
plt.show()

In [ ]:
print( "The best validation accuracy was", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

# Decision Tree

#### splitting the data

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.3, random_state=3)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Validation set:', X_eval.shape,  y_eval.shape)

#### training

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decisionTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
decisionTree.fit(X_train,y_train)

#### predicting

In [ ]:
y_pred = decisionTree.predict(X_eval)
y_pred[0:5]

#### evaluating

In [ ]:
print (classification_report(y_eval, y_pred))

In [ ]:
print("Train set Accuracy: ", accuracy_score(y_train, decisionTree.predict(X_train)))
print("Validation set Accuracy: ", accuracy_score(y_eval, y_pred))

#### Confusion Matrix

In [ ]:
confusion_matrix(y_eval, y_pred)

#### finding the best depth

In [ ]:
maxd=10
mean_acc = np.zeros((maxd-1))
for n in range(1,maxd):
    decisionTree = DecisionTreeClassifier(criterion='entropy',max_depth=n).fit(X_train,y_train)
    y_pred=decisionTree.predict(X_eval)
    mean_acc[n-1]= accuracy_score(y_eval, y_pred)
mean_acc[0:5]

In [ ]:
print( "The best validation accuracy was", mean_acc.max(), "with maxdepth=", mean_acc.argmax()+1) 

# Support Vector Machine

#### splitting the data

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Validation set:', X_eval.shape,  y_eval.shape)

#### training

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train) 

#### predicting

In [ ]:
y_pred = clf.predict(X_eval)
y_pred [0:5]

#### evaluating

In [ ]:
print (classification_report(y_eval, y_pred))

In [ ]:
print("Train set Accuracy: ", accuracy_score(y_train, clf.predict(X_train)))
print("Validation set Accuracy: ", accuracy_score(y_eval, y_pred))
print("Avg F1-score: %.4f" % f1_score(y_eval, y_pred, average='weighted'))
print("Jaccard score: %.4f" % jaccard_score(y_eval, y_pred, pos_label="PAIDOFF"))

#### Confusion Matrix

In [ ]:
confusion_matrix(y_eval, y_pred)

# Logistic Regression

#### splitting the data

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.3, random_state=3)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Validation set:', X_eval.shape,  y_eval.shape)

#### training

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
lr

#### predicting

In [ ]:
y_pred = lr.predict(X_eval)
y_pred [0:5]

In [ ]:
y_pred_prob = lr.predict_proba(X_eval)
y_pred_prob [0:5]

#### evaluating

In [ ]:
print (classification_report(y_eval, y_pred))

In [ ]:
print("Train set Accuracy: ", accuracy_score(y_train, lr.predict(X_train)))
print("Validation set Accuracy: ", accuracy_score(y_eval, y_pred))
print("Avg F1-score: %.4f" % f1_score(y_eval, y_pred, average='weighted'))
print("Jaccard score: %.4f" % jaccard_score(y_eval, y_pred, pos_label="PAIDOFF"))

#### Confusion Matrix

In [ ]:
confusion_matrix(y_eval, y_pred)

# Model Evaluation using Test set

In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation 

In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

#### Data preprocessing

In [ ]:
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_df.head()

#### Feature selection

In [ ]:
Feature_test = test_df[['Principal','terms','age','Gender','weekend']]
Feature_test = pd.concat([Feature_test,pd.get_dummies(test_df['education'])], axis=1)
Feature_test.drop(['Master or Above'], axis = 1,inplace=True)
Feature_test.head()

#### Normalize data

In [ ]:
X_test = preprocessing.StandardScaler().fit(Feature_test).transform(Feature_test)
X_test[0:5]

In [ ]:
y_test = test_df['loan_status'].values
y_test[0:5]

#### Modeling and training

In [ ]:
knn_test = KNeighborsClassifier(n_neighbors = 7).fit(X_test,y_test)
dt_test = DecisionTreeClassifier(criterion="entropy", max_depth = 4).fit(X_test,y_test)
clf_test = svm.SVC(kernel='rbf').fit(X_test,y_test)
lr_test = LogisticRegression(C=0.01, solver='liblinear').fit(X_test,y_test)

#### Predicting

In [ ]:
jaccard_knn = jaccard_score(y_test, knn_test.predict(X_test), pos_label="PAIDOFF")
jaccard_dt = jaccard_score(y_test, dt_test.predict(X_test), pos_label="PAIDOFF")
jaccard_clf = jaccard_score(y_test, clf_test.predict(X_test), pos_label="PAIDOFF")
jaccard_lr = jaccard_score(y_test, lr_test.predict(X_test), pos_label="PAIDOFF")

In [ ]:
f1_knn = f1_score(y_test, knn_test.predict(X_test), pos_label="PAIDOFF")
f1_dt = f1_score(y_test, dt_test.predict(X_test), pos_label="PAIDOFF")
f1_clf = f1_score(y_test, clf_test.predict(X_test), pos_label="PAIDOFF")
f1_lr = f1_score(y_test, lr_test.predict(X_test), pos_label="PAIDOFF")

In [ ]:
loss_lr = log_loss(y_test, lr_test.predict_proba(X_test))

#### Results

In [ ]:
results=pd.DataFrame()

In [ ]:
results['Algorithm'] = (['KNN','Decision Tree','SVM','LogisticRegression'])
results['Jaccard'] = ([jaccard_knn,jaccard_dt,jaccard_clf,jaccard_lr])
results['F1-score'] = ([f1_knn,f1_dt,f1_clf,f1_lr])
results['LogLoss'] = (['NA','NA','NA',loss_lr])
results

# Report
You should be able to report the accuracy of the built model using different evaluation metrics:

| Algorithm          | Jaccard | F1-score | LogLoss |
|--------------------|---------|----------|---------|
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>